## Chapter 6

함수 객체를 이용하면 코드를 훨씬 보기 좋게 만들 수 있다. 특히 함수를 일급 객체로 취급하는 파이썬에서는 함수 객체를 이용해 반복되는 코드의 상당 부분을 줄일 수 있다.

이를테면, 함수 객체를 이용해 디자인 패턴 중 '전략 패턴'을 크게 단순화할 수 있다. 디자인 패턴은 물론 좋은 방법론이지만 언제나 맹목적으로 따라야 할 대상은 아니다.

+ 고전적인 전략 패턴

먼저 '디자인 패턴' 에서의 전략 패턴은 다음과 같이 설명된다.

>일련의 알고리즘을 정의하고 각각을 하나의 클래스 안에 넣어서 교체하기 쉽게 만든다. 전략을 이용하면 사용하는 클라이언트에 따라 알고리즘을 독립적으로 변경할 수 있다.

이를테면 다음과 같은 코드가 전략 패턴의 사례이다. 고객의 속성이나 주문한 상품에 따라 해당하는 조건의 할인율을 계산해서 할인을 적용하는 코드이다.

In [ ]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer=namedtuple('Customer', 'name fidelity')


class LineItem: #구입한 물건
    def __init__(self, product, quantity, price):
        self.product=product
        self.quantity=quantity
        self.price=price

    def total(self):
        return self.price*self.quantity


class Order:
    def __init__(self, customer, cart, promotion=None): #Customer 튜플 받음
        self.customer=customer
        self.cart=list(cart) #LineItem들이 들어 있음
        self.promotion=promotion #적용되는 할인. 기본 할인은 없다

    def total(self):
        if not hasattr(self, '__total'):
            self.__total=sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount=0
        else:
            discount=self.promotion.discount(self) #깎이는 비용
        return self.total()-discount

    def __repr__(self):
        fmt='<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


class Promotion(ABC): #abstract class inherit
    def discount(self, order): #할인액을 구체적 숫자로 변환
        """상속해서 오버라이딩하는 것이 목적인 추상 클래스. 각각의 조건에 따라 할인을 적용하게 된다"""


class FidelityPromo(Promotion): #충성도 1000 이상이면 5% 전체할인 적용
    def discount(self, order):
        return order.total()*0.05 if order.customer.fidelity>=1000 else 0


class BulkItemPromo(Promotion): #같은 상품 20개 이상 구입시 그 상품군 10% 할인 적용
    def discount(self, order):
        dc=0
        for item in order.cart:
            if item.quantity>=20:
                dc+=item.total()*0.1
        return dc


class LargeOrderPromo(Promotion): #10종류 이상 상품 구입시 7% 전체할인 적용
    def discount(self, order):
        distinct_items={item.product for item in order.cart} #set을 이용해 상품 종류 계산
        if len(distinct_items)>=10:
            return order.total*0.07
        return 0


Joe=Customer('John Doe',0)
Ann=Customer('Ann Smith',1100)
cart=[LineItem('banana', 4, 0.5),
      LineItem('apple', 25, 1.5),
      LineItem('melon', 5, 5.0)]
print(Order(Joe, cart, BulkItemPromo()))
print(Order(Ann, cart, FidelityPromo()))




하지만 함수 객체를 사용하면 이를 더 깔끔하게 짤 수 있다. 추상 클래스를 사용해서 상속받는 코드를 짤 필요가 없는 것이다. 각각의 할인 전략은 discount()라는 메서드를 가진 하나의 클래스인데, 이 전략을 그냥 함수로 변경하면 추상 클래스를 쓸 필요가 없다. 복잡한 전략이라면 추상 클래스를 사용해서 코드를 짜는 게 좋은 전략일 수 있겠지만 이런 간단한 코드에 굳이 추상 클래스를 사용할 필요가 없다.

In [ ]:
#함수를 이용해 간단하게 리팩토링

from abc import ABC, abstractmethod
from collections import namedtuple

Customer=namedtuple('Customer', 'name fidelity')


class LineItem: #구입한 물건
    def __init__(self, product, quantity, price):
        self.product=product
        self.quantity=quantity
        self.price=price

    def total(self):
        return self.price*self.quantity


class Order:
    def __init__(self, customer, cart, promotion=None): #Customer 튜플 받음
        self.customer=customer
        self.cart=list(cart) #LineItem들이 들어 있음
        self.promotion=promotion #적용되는 할인. 기본 할인은 없다

    def total(self):
        if not hasattr(self, '__total'):
            self.__total=sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount=0
        else:
            discount=self.promotion(self) #깎이는 비용
        return self.total()-discount

    def __repr__(self):
        fmt='<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

def fidelity_promo(order):
  return order.total()*0.05 if order.customer.fidelity>=1000 else 0

def bulkitem_promo(order):
  dc=0
  for item in order.cart:
      if item.quantity>=20:
          dc+=item.total()*0.1
  return dc

def largeorder_promo(self, order):
    distinct_items={item.product for item in order.cart} #set을 이용해 상품 종류 계산
    if len(distinct_items)>=10:
        return order.total*0.07
    return 0

Joe=Customer('John Doe',0)
Ann=Customer('Ann Smith',1100)
cart=[LineItem('banana', 4, 0.5),
      LineItem('apple', 25, 1.5),
      LineItem('melon', 5, 5.0)]
print(Order(Joe, cart, bulkitem_promo))
print(Order(Ann, cart, fidelity_promo))




<Order total: 64.50 due: 60.75>
<Order total: 64.50 due: 61.27>


위 코드와 같이 클래스가 함수 객체를 인수로 직접 받도록 하면, 굳이 추상 클래스를 구현하고 클래스 내부에 할인 함수를 따로 만들 필요가 없다. 코드가 매우 짧아졌고 추상 클래스가 없어져서 직관적으로 변한 것을 알 수 있다. 또한 Order 클래스를 사용 시 _promo함수 객체를 바로 가져다 쓸 수 있기 때문에 할인 전략 객체를 일일이 만들어 줄 필요가 없다.

+ 메타 전략의 문제

그런데 문제가 있다. 적용 가능한 할인 전략 중에 가장 좋은 것을 선택한다든지 하는 '메타 전략' 을 만들어야 할 경우 어떻게 함수 객체를 활용할 것인가? 물론, 가장 단순한 해법으로 다음과 같은 함수 객체 리스트를 만드는 것을 생각할 수 있다. 다음 코드의 best_promo 함수는 할인 전략 중 가장 많은 할인율이 적용되는 할인을 적용한다.

In [ ]:
promos=[fidelity_promo, bulkitem_promo, largeorder_promo]

def best_promo(order):
  return max(promo(order) for promo in promos)

+ 함수 추가 시의 대응 전략

그런데 문제는 새로운 할인 함수가 추가될 때의 대응 전략이다. 새로운 할인 함수를 best_promo 함수가 고려해 줄 수 있도록 해야 한다. 물론 새로운 함수를 promos에 넣어 주면 된다. 하지만 이는 긴 코드를 다룰 때 실수를 하기 쉬운 방식이다. 좀더 좋은 방법이 있다. 

+ globals() 의 활용

첫번째는 globals() 내장 함수를 사용하는 것이다. globals는 모듈의 전역 심벌 테이블을 나타내는 딕셔너리를 반환한다. 그것을 이용하면 다음과 같은 코드를 짤 수 있다. 또는 best_promo를 제외한 할인 함수들을 모두 별도의 모듈에 넣고 globals()를 이용해 모두 모으는 방식도 있다.

In [1]:
promos=[globals()[name] for name in globals()
if name.endswith('_promo') and name!='best_promo']

def best_promo(order):
  return max(promo(order) for promo in promos)

+ inspect 모듈의 사용

또는 inspect 모듈의 getmembers함수를 사용하는 방법도 있는데 이는 inspect.isfunction 함수를 사용 시 모듈의 함수들을 전부 조사해서 promos 리스트에 넣는다. 만약 그럴 경우 함수이기만 하면 promos 리스트에 들어가게 되므로 굳이 함수 이름이 _promo로 끝날 필요가 없게 된다. 

이렇게 특정한 역할을 하는 함수를 따로 모으는 일이 필요할 때, 데커레이터를 사용하면 더욱 명시적으로 처리할 수 있는데 이는 7장에서 다룬다.

이런 식으로 함수 객체를 이용시 객체지향의 디자인 패턴들을 상당히 많이 대체할 수 있고 또 더 가독성 좋은 코드를 짤 수도 있다. 그리고 `__call__` 함수에서 그냥 함수 객체를 호출할 시에 명령 패턴도 대체할 수 있다. 데커레이터를 사용시 훨씬 더 많은 대체가 가능하다. 7장에서는 이 할인 코드를 더 리팩토링하며 데커레이터의 사용을 배울 것이다. 